# Εργαστήριο 8
Όπως και στα προηγούμενα εργαστήρια συνεχίζουμε στο περιβάλλον του [online chisel bootcamp](https://mybinder.org/v2/gh/freechipsproject/chisel-bootcamp/master).

Πριν ξεκινήσετε, εκτελέστε τα επόμενα 2 κελιά:

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Προαπαιτούμενα modules
Για την υλοποίηση της τρέχουσας άσκησης θα χρειαστείτε **από τα προηγούμενα εργαστήρια** τα εξής:

* Το παραμετρικό module `RegisterFile(register_width: Int, register_number: Int)`

* Το παραμετρικό module `Alu(n: Int)`


 **Αντιγράψτε** τα πιο πάνω modules στα επόμενα 2 κελιά.

In [ ]:
class RegisterFile(register_width: Int, register_number: Int) extends Module {
  
  // συμπληρώστε τον κώδικα από το δικό σας module
    
}

In [ ]:
class Alu(n: Int) extends Module {

  // συμπληρώστε τον κώδικα από το δικό σας module

}

## Άσκηση
Εδώ υλοποιείτε μια πρώτη εκδοχή του module `DataPath` που συνδυάζει `RegisterFile` με `register_number` καταχωρητές και εύρος λέξης `word_width` bits και αντίστοιχα `Alu` των `word_width` bits επίσης, για την υλοποίηση ενός κυκλώματος που θα μπορεί

* να εκτελεί πράξεις μεταξύ δύο καταχωρητών και 
* να αποθηκεύει το αποτέλεσμα σε καταχωρητή.

Η υλοποίηση πρέπει να γίνει σύμφωνα με το **επόμενο σχήμα**:

![datapath1.png](https://mixstef.github.io/courses/comparch/labimg/datapath1.png)


Στο σχήμα, τα νέα σήματα είναι το `alu_a_sel` και το `im` (για τα υπόλοιπα σήματα ανατρέξτε στο αντίστοιχο εργαστήριο του alu ή του registerfile).

Η λειτουργία του σήματος `alu_a_sel` (εύρους 1 bit) είναι η ακόλουθη:

* Όταν το `alu_a_sel` είναι 0, τότε στην είσοδο a της alu εισάγεται η λέξη της εισόδου `im` (με εύρος `word_width` bits).

* Όταν το `alu_a_sel` είναι 1, τότε στην είσοδο a της alu εισάγεται η έξοδος `out_a` του register file.


Στο επόμενο κελί υπάρχει ο βασικός σκελετός κώδικα για το module `DataPath`. Παρατηρήστε πώς εισάγουμε instances των `RegisterFile` και `Alu` για χρήση μέσα στο module:

~~~scala
val regfile = Module(new RegisterFile(word_width,register_number))
val alu = Module(new Alu(word_width))
~~~

Εάν ορίσετε τα δύο instances όπως πάνω, τότε μπορείτε να δημιουργήσετε συνδέσεις όπως π.χ.:

~~~scala
regfile.io.in := alu.io.out  // η είσοδος in του regfile συνδέεται στην έξοδο out του alu

alu.io.sel := io.sel  // η είσοδος sel του alu συνδέεται στην είσοδο sel του κυρίως module (DataPath)
~~~

Συμπληρώστε ό,τι χρειάζεται στο επόμενο κελί, έτσι ώστε να παραχθεί το κύκλωμα του προηγούμενου σχήματος.

**Υπόδειξη:** Ο πολυπλέκτης (MUX) του σχήματος που επιλέγει μία από δύο εισόδους με βάση το σήμα `alu_a_sel` μπορεί να υλοποιηθεί με μια δομή `when`.

In [ ]:
class DataPath(word_width: Int, register_number: Int) extends Module {
  val io = IO(new Bundle {

    // συμπληρώστε τις εισόδους και εξόδους σύμφωνα με το σχήμα
    
    // διατηρήστε τα ονόματα των σημάτων!
      
  })
  
  // διατηρήστε τις δύο επόμενες γραμμές
  val regfile = Module(new RegisterFile(word_width,register_number))
  val alu = Module(new Alu(word_width))

  // συμπληρώστε υλοποιώντας τις συνδέσεις του σχήματος

}

## Δοκιμή κυκλώματος datapath
Μελετήστε προσεκτικά τι κάνει ο επόμενος δοκιμαστικός κώδικας:

Η μεταβλητή `cmdbits` είναι μια λίστα (List) από Maps.
* Στη Scala, List είναι μια ακολουθία που μπορούμε να διασχίσουμε από την μια άκρη στην άλλη (όχι όμως να πάρουμε το i-οστό στοιχείο). Εδώ έχουμε ένα `for (bits <- cmdbits) {}` που σε κάθε επανάληψη αναθέτει στη μεταβλητή `bits` ένα-ένα τα περιεχόμενα της λίστας `cmdbits`.

* Map στη Scala είναι μια δομή (κλειδιού/τιμής). Στον κώδικα που ακολουθεί παίρνουμε τις τιμές από κάθε Map και οδηγούμε ανάλογα τα σήματα εισόδου του DataPath. Στη συνέχεια δίνουμε έναν παλμό ρολογιού και τυπώνουμε την έξοδο `results`. Αυτό επαναλαμβάνεται για κάθε Map της λίστας.

Όπως φαίνεται, στον επόμενο κώδικα εκτελούνται 2 "λειτουργίες" (micro-instructions). **Μπορείτε να βρείτε ποιες είναι αυτές;**

In [ ]:
test(new DataPath(8,8)) { c =>
  // array of command bits (micro-instructions)
  val cmdbits = List(Map("READ_SEL_A" -> 0.U,    // λειτουργία Α
                         "READ_SEL_B" -> 0.U,
                         "ALU_A_SEL" -> 0.U,
                         "IM" -> 33.U,
                         "SEL" -> "b01".U,
                         "SUB" -> 0.U,
                         "WRITE_SEL" -> 1.U),
                     Map("READ_SEL_A" -> 0.U,    // λειτουργία Β
                         "READ_SEL_B" -> 1.U,
                         "ALU_A_SEL" -> 1.U,
                         "IM" -> 0.U,
                         "SEL" -> "b01".U,
                         "SUB" -> 0.U,
                         "WRITE_SEL" -> 0.U)
                    )
      
  for (bits <- cmdbits) {
    c.io.read_sel_a.poke(bits("READ_SEL_A"))
    c.io.read_sel_b.poke(bits("READ_SEL_B"))
    c.io.alu_a_sel.poke(bits("ALU_A_SEL"))
    c.io.im.poke(bits("IM"))
    c.io.sel.poke(bits("SEL"))
    c.io.sub.poke(bits("SUB"))
    c.io.write_sel.poke(bits("WRITE_SEL"))
    println(c.io.results.peek())
    c.clock.step()
  }  
}

### Δοκιμάστε κι εσείς
Μπορείτε να τροποποιήσετε το περιεχόμενο της λίστας `cmdbits` (προσθέτοντας κι άλλα Maps, αν χρειάζεται), έτσι ώστε να υλοποιηθούν οι εξής λειτουργίες:
~~~
r1 = 33
r2 = 34
r3 = r1 + r2
~~~

**Θυμηθείτε ότι μπορείτε να τροποποιήσετε στον κώδικα μόνο το περιεχόμενο της λίστας `cmdbits`!**

In [ ]:
test(new DataPath(8,8)) { c =>
  // array of command bits (micro-instructions)
  val cmdbits = List(Map("READ_SEL_A" -> ..συμπληρώστε..,    // r1 = 33
                         "READ_SEL_B" -> ..συμπληρώστε..,
                         "ALU_A_SEL" -> ..συμπληρώστε..,
                         "IM" -> ..συμπληρώστε..,
                         "SEL" -> ..συμπληρώστε..,
                         "SUB" -> ..συμπληρώστε..,
                         "WRITE_SEL" -> ..συμπληρώστε..),
                     Map(..συμπληρώστε..,    // r2 = 34
                        ),
                     Map(..συμπληρώστε..,    // r3 = r1 + r2
                        )
                    )
      
  for (bits <- cmdbits) {
    c.io.read_sel_a.poke(bits("READ_SEL_A"))
    c.io.read_sel_b.poke(bits("READ_SEL_B"))
    c.io.alu_a_sel.poke(bits("ALU_A_SEL"))
    c.io.im.poke(bits("IM"))
    c.io.sel.poke(bits("SEL"))
    c.io.sub.poke(bits("SUB"))
    c.io.write_sel.poke(bits("WRITE_SEL"))
    println(c.io.results.peek())
    c.clock.step()
  }  
}

## Τελειώνοντας...
Κατεβάσετε και πάρτε μαζί σας το σημερινό notebook για να το έχετε στο αρχείο σας.

**Δεν ανήκει στα παραδοτέα του εργαστηρίου.**